# Transformações e Padronizações – Camada Trusted

Este notebook realiza a transformação dos dados provenientes da camada RAW, aplicando regras de limpeza, normalização, padronização de tipos, renomeação de colunas e modelagem estrutural. O objetivo é gerar datasets confiáveis, prontos para análises, integração e futura carga na camada Refined ou Gold.

As tabelas tratadas incluem: **empresas**, **estabelecimentos**, **sócios**, **simples_nacional**, **CNAE**, **municípios**, **natureza_jurídica**, **motivos** e **países**.

In [64]:
# Para iniciar a seção spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DecimalType, IntegerType, LongType
from pyspark.sql import functions as f
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [2]:
builder = SparkSession.builder \
    .appName("App Preparação RAW") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g")

spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/24 12:10:05 WARN Utils: Your hostname, BRALSOFT42, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/24 12:10:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/wilcb/projeto_data_warehouse/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/wilcb/.ivy2.5.2/cache
The jars for the packages stored in: /home/wilcb/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8155c6e2-d9bb-4f67-b335-2cb08e72b698;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 221ms :: artifacts dl 11ms
	:: mod

## Funções auxiliares

In [3]:
def null_count(df: DataFrame) -> None:
    """
    Conta e exibe a quantidade de valores nulos por coluna em um DataFrame do PySpark.

    A função percorre todas as colunas do DataFrame fornecido e calcula, para cada uma, 
    a quantidade de valores nulos (`NULL`). O resultado é exibido diretamente no console 
    por meio do método `.show()`.

    Args:
        df (DataFrame): DataFrame a ser inspecionado.

    Returns:
        None: A função apenas imprime o resultado no console.
    """
    try:
        # Cria uma lista de expressões que somam os valores nulos por coluna
        nulls = [
            f.sum(f.col(c).isNull().cast("int")).alias(c)
            for c in df.columns
        ]

        # Exibe os totais de nulos por coluna
        df.select(nulls).show()
    except Exception as e:
        print(f"[ERRO] Falha na busca de dados nulos: {e}")
        raise

## Leitura do DeltaTable na camada RAW

In [4]:
try:
    deltaTable_estabelecimentos_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/estabelecimentos")
    deltaTable_empresas_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/empresas")
    deltaTable_municipios_rf_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/municipios_rf")
    deltaTable_municipios_ibge_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/municipios_ibge")
    deltaTable_cnae_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/cnae")
    deltaTable_qualificacao_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/qualificacoes")
    deltaTable_natureza_juridica_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/natureza_juridica")
    deltaTable_simples_nacional_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/simples_nacional")
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

In [5]:
try:
    df_estabelecimentos_raw = deltaTable_estabelecimentos_raw.toDF()
    df_empresas_raw = deltaTable_empresas_raw.toDF()
    df_municipios_rf_raw = deltaTable_municipios_rf_raw.toDF()
    df_municipios_ibge_raw = deltaTable_municipios_ibge_raw.toDF()
    df_cnae_raw = deltaTable_cnae_raw.toDF()
    df_qualificacao_raw = deltaTable_qualificacao_raw.toDF()
    df_natureza_juridica_raw = deltaTable_natureza_juridica_raw.toDF()
    df_simples_nacional_raw = deltaTable_simples_nacional_raw.toDF()
except Exception as e:
    print(f"[ERRO] Falha na transformação para DataFrame: {e}")
    raise

## TRUSTED - Municipios IBGE

### Etapa 1: Leitura da camada RAW

In [23]:
df_municipios_ibge_raw.printSchema()
df_municipios_ibge_raw.show(5, truncate=False)

root
 |-- id: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- microrregiao: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- mesorregiao: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- nome: string (nullable = true)
 |    |    |-- UF: struct (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- sigla: string (nullable = true)
 |    |    |    |-- nome: string (nullable = true)
 |    |    |    |-- regiao: struct (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- sigla: string (nullable = true)
 |    |    |    |    |-- nome: string (nullable = true)
 |-- regiao-imediata: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- regiao-intermediaria: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- nome

### Etapa 2: Transformação dos dados

- Flatten dos campos aninhados

- Renomeação das colunas

- Padronização de nomes para o padrão Trusted

In [29]:
try:
    df_municipios_ibge_trusted = df_municipios_ibge_raw.select(
        f.col("id").alias("id_municipio"),
        f.col("nome").alias("nome_municipio"),

        f.col("microrregiao.id").alias("id_microrregiao"),
        f.col("microrregiao.nome").alias("nome_microrregiao"),

        f.col("microrregiao.mesorregiao.id").alias("id_mesorregiao"),
        f.col("microrregiao.mesorregiao.nome").alias("nome_mesorregiao"),

        f.col("microrregiao.mesorregiao.UF.id").alias("id_uf"),
        f.col("microrregiao.mesorregiao.UF.sigla").alias("sigla_uf"),
        f.col("microrregiao.mesorregiao.UF.nome").alias("nome_uf"),

        f.col("microrregiao.mesorregiao.UF.regiao.id").alias("id_regiao"),
        f.col("microrregiao.mesorregiao.UF.regiao.sigla").alias("sigla_regiao"),
        f.col("microrregiao.mesorregiao.UF.regiao.nome").alias("nome_regiao"),
        f.col("data_ingestao")
    )
except Exception as e:
    print(f"[ERRO] Falha na execução: {e}")
    raise

In [30]:
df_municipios_ibge_trusted.show(5, truncate=False)
df_municipios_ibge_trusted.printSchema()

+------------+---------------------+---------------+-----------------+--------------+-----------------+-----+--------+--------+---------+------------+-----------+--------------------------+
|id_municipio|nome_municipio       |id_microrregiao|nome_microrregiao|id_mesorregiao|nome_mesorregiao |id_uf|sigla_uf|nome_uf |id_regiao|sigla_regiao|nome_regiao|data_ingestao             |
+------------+---------------------+---------------+-----------------+--------------+-----------------+-----+--------+--------+---------+------------+-----------+--------------------------+
|1100015     |Alta Floresta D'Oeste|11006          |Cacoal           |1102          |Leste Rondoniense|11   |RO      |Rondônia|1        |N           |Norte      |2025-07-23 23:12:11.297894|
|1100023     |Ariquemes            |11003          |Ariquemes        |1102          |Leste Rondoniense|11   |RO      |Rondônia|1        |N           |Norte      |2025-07-23 23:12:11.297894|
|1100031     |Cabixi               |11008         

### Etapa 3: Escrita na camada Trusted (TRS)

- Particionado por: sigla_uf

In [31]:
try:
    df_municipios_ibge_trusted.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("sigla_uf") \
        .save("../TRS/municipios_ibge")
    print("Dados salvo!")
except Exception as e:
    print(f"[ERRO] Falha na escrita do deltaTable: {e}")
    raise

Dados salvo!


### Etapa 4: Ajuste e Validação

- Releitura para verificação

In [32]:
try:
    deltaTable_municipios_ibge_trs = DeltaTable.forPath(spark, "../TRS/municipios_ibge")
    df_municipios_ibge_trs = deltaTable_municipios_ibge_trs.toDF()
    df_municipios_ibge_trs.show(5, truncate=False)
    df_municipios_ibge_trs.printSchema()
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

+------------+---------------------+---------------+----------------------------+--------------+----------------+-----+--------+--------+---------+------------+-----------+--------------------------+
|id_municipio|nome_municipio       |id_microrregiao|nome_microrregiao           |id_mesorregiao|nome_mesorregiao|id_uf|sigla_uf|nome_uf |id_regiao|sigla_regiao|nome_regiao|data_ingestao             |
+------------+---------------------+---------------+----------------------------+--------------+----------------+-----+--------+--------+---------+------------+-----------+--------------------------+
|2100055     |Açailândia           |21009          |Imperatriz                  |2102          |Oeste Maranhense|21   |MA      |Maranhão|2        |NE          |Nordeste   |2025-07-23 23:12:11.297894|
|2100105     |Afonso Cunha         |21016          |Coelho Neto                 |2104          |Leste Maranhense|21   |MA      |Maranhão|2        |NE          |Nordeste   |2025-07-23 23:12:11.297894|


- Validação de nulos (colunas)

In [47]:
try:
    null_count(df_municipios_ibge_trs)
except Exception as e:
    print(f"[ERRO] Falha na busca de dados nulos: {e}")
    raise

+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+-------------+
|id_municipio|nome_municipio|id_microrregiao|nome_microrregiao|id_mesorregiao|nome_mesorregiao|id_uf|sigla_uf|nome_uf|id_regiao|sigla_regiao|nome_regiao|data_ingestao|
+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+-------------+
|           0|             0|              0|                0|             0|               0|    0|       0|      0|        0|           0|          0|            0|
+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+-------------+



In [34]:
df_municipios_ibge_trs.select("*") \
    .filter(df_municipios_ibge_trs["id_microrregiao"].isNull()) \
    .show(truncate=False)

+------------+----------------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+--------------------------+
|id_municipio|nome_municipio        |id_microrregiao|nome_microrregiao|id_mesorregiao|nome_mesorregiao|id_uf|sigla_uf|nome_uf|id_regiao|sigla_regiao|nome_regiao|data_ingestao             |
+------------+----------------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+--------------------------+
|5101837     |Boa Esperança do Norte|NULL           |NULL             |NULL          |NULL            |NULL |NULL    |NULL   |NULL     |NULL        |NULL       |2025-07-23 23:12:11.297894|
+------------+----------------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+--------------------------+



- Correção de valores nulos pontuais

In [35]:
try:
    # Remoção de NULL's
    deltaTable_municipios_ibge_trs.update(
        condition=f.col("id_municipio") == "5101837",
        set={

            "id_uf": f.lit("51"),
            "sigla_uf": f.lit("MT"),
            "nome_uf": f.lit("Mato Grosso"),
            "id_regiao": f.lit("5"),
            "sigla_regiao": f.lit("CO"),
            "nome_regiao": f.lit("Centro-Oeste"),
        }
    )

    print("Coluna atualizada com sucesso!")
except Exception as e:
    print(f"[ERRO] Falha ao tentar efetuar update: {e}")
    raise

25/07/24 11:03:34 WARN UpdateCommand: Could not validate number of records due to missing statistics.


Coluna atualizada com sucesso!


In [36]:
df_municipios_ibge_trs.select("*") \
    .filter(df_municipios_ibge_trs["id_microrregiao"].isNull()) \
    .show(truncate=False)

+------------+----------------------+---------------+-----------------+--------------+----------------+-----+--------+-----------+---------+------------+------------+--------------------------+
|id_municipio|nome_municipio        |id_microrregiao|nome_microrregiao|id_mesorregiao|nome_mesorregiao|id_uf|sigla_uf|nome_uf    |id_regiao|sigla_regiao|nome_regiao |data_ingestao             |
+------------+----------------------+---------------+-----------------+--------------+----------------+-----+--------+-----------+---------+------------+------------+--------------------------+
|5101837     |Boa Esperança do Norte|NULL           |NULL             |NULL          |NULL            |51   |MT      |Mato Grosso|5        |CO          |Centro-Oeste|2025-07-23 23:12:11.297894|
+------------+----------------------+---------------+-----------------+--------------+----------------+-----+--------+-----------+---------+------------+------------+--------------------------+



In [37]:
try:
    deltaTable_municipios_ibge_trs.update(
        condition=f.col("id_municipio") == "5101837",
        set={
            "id_microrregiao": f.lit("000000"),
            "nome_microrregiao": f.lit("desconhecido"),
            "id_mesorregiao": f.lit("0000"),
            "nome_mesorregiao": f.lit("desconhecido"),
        }
    )
    print("Campos regionais preenchidos com valor padrão 'desconhecido'.")
except Exception as e:
    print(f"[ERRO] Falha ao preencher campos com valor padrão: {e}")
    raise

25/07/24 11:08:32 WARN UpdateCommand: Could not validate number of records due to missing statistics.


Campos regionais preenchidos com valor padrão 'desconhecido'.


In [38]:
df_municipios_ibge_trs.select("*") \
    .filter(df_municipios_ibge_trs["id_microrregiao"].isNull()) \
    .show(truncate=False)

+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+-------------+
|id_municipio|nome_municipio|id_microrregiao|nome_microrregiao|id_mesorregiao|nome_mesorregiao|id_uf|sigla_uf|nome_uf|id_regiao|sigla_regiao|nome_regiao|data_ingestao|
+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+-------------+
+------------+--------------+---------------+-----------------+--------------+----------------+-----+--------+-------+---------+------------+-----------+-------------+



## TRUSTED - Natureza Jurídica

### Etapa 1: Leitura da camada RAW

In [39]:
df_natureza_juridica_raw.printSchema()
df_natureza_juridica_raw.show(5, truncate=False)

root
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- natureza_juridica: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+------------------------+------------------------------------------+--------------------------+
|codigo_natureza_juridica|natureza_juridica                         |data_ingestao             |
+------------------------+------------------------------------------+--------------------------+
|0000                    |Natureza Jurídica não informada           |2025-07-24 09:53:45.817935|
|3271                    |Órgão de Direção Local de Partido Político|2025-07-24 09:53:45.817935|
|3280                    |Comitê Financeiro de Partido Político     |2025-07-24 09:53:45.817935|
|3298                    |Frente Plebiscitária ou Referendária      |2025-07-24 09:53:45.817935|
|3301                    |Organização Social (OS)                   |2025-07-24 09:53:45.817935|
+------------------------+---------------------------------------

In [42]:
df_natureza_juridica_raw = df_natureza_juridica_raw.withColumnRenamed("natureza_juridica", "descricao_natureza_juridica")
df_natureza_juridica_raw.printSchema()

root
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- descricao_natureza_juridica: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)



### Etapa 2: Escrita na camada Trusted (TRS)

In [43]:
try:
    df_natureza_juridica_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/natureza_juridica")
except Exception as e:
    print(f"[ERRO] Falha ao salvar dados no DeltaTable: {e}")
    raise

### Etapa 3: Ajuste e Validação

In [44]:
try:
    deltaTable_natureza_juridica_trs = DeltaTable.forPath(spark, "../TRS/natureza_juridica")
    df_natureza_juridica_trs: DataFrame = deltaTable_natureza_juridica_trs.toDF()
    df_natureza_juridica_trs.printSchema()
    df_natureza_juridica_trs.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura: {e}")
    raise

root
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- descricao_natureza_juridica: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+------------------------+------------------------------------------+--------------------------+
|codigo_natureza_juridica|descricao_natureza_juridica               |data_ingestao             |
+------------------------+------------------------------------------+--------------------------+
|0000                    |Natureza Jurídica não informada           |2025-07-24 09:53:45.817935|
|3271                    |Órgão de Direção Local de Partido Político|2025-07-24 09:53:45.817935|
|3280                    |Comitê Financeiro de Partido Político     |2025-07-24 09:53:45.817935|
|3298                    |Frente Plebiscitária ou Referendária      |2025-07-24 09:53:45.817935|
|3301                    |Organização Social (OS)                   |2025-07-24 09:53:45.817935|
+------------------------+-----------------------------

In [49]:
try:
    null_count(df_natureza_juridica_trs)
except Exception as e:
    print(f"[ERRO] Falha na busca de dados nulos: {e}")
    raise

+------------------------+---------------------------+-------------+
|codigo_natureza_juridica|descricao_natureza_juridica|data_ingestao|
+------------------------+---------------------------+-------------+
|                       0|                          0|            0|
+------------------------+---------------------------+-------------+



## TRUSTED - CNAE

### Etapa 1: Leitura da camada RAW

In [50]:
df_cnae_raw.printSchema()
df_cnae_raw.show(5, truncate=False)

root
 |-- codigo_cnae: string (nullable = true)
 |-- descricao_cnae: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-----------+---------------------------------------------------------+--------------------------+
|codigo_cnae|descricao_cnae                                           |data_ingestao             |
+-----------+---------------------------------------------------------+--------------------------+
|0111301    |Cultivo de arroz                                         |2025-07-24 09:49:47.363057|
|0111302    |Cultivo de milho                                         |2025-07-24 09:49:47.363057|
|0111303    |Cultivo de trigo                                         |2025-07-24 09:49:47.363057|
|0111399    |Cultivo de outros cereais não especificados anteriormente|2025-07-24 09:49:47.363057|
|0112101    |Cultivo de algodão herbáceo                              |2025-07-24 09:49:47.363057|
+-----------+----------------------------------------------------

### Etapa 2: Escrita na camada Trusted (TRS)

In [ ]:
try:
    df_cnae_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/cnae")

    print("Salvo com sucesso!")
except Exception as e:
    print(f"[ERRO] Falha ao gravar DeltaTable: {e}")
    raise

### Etapa 3: Ajuste e Validação

In [52]:
try:
    deltaTable_cnae: DeltaTable = DeltaTable.forPath(spark, "../TRS/cnae")
    df_cnae_trusted: DataFrame = deltaTable_cnae.toDF()
    df_cnae_trusted.printSchema()
    df_cnae_trusted.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

root
 |-- codigo_cnae: string (nullable = true)
 |-- descricao_cnae: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-----------+---------------------------------------------------------+--------------------------+
|codigo_cnae|descricao_cnae                                           |data_ingestao             |
+-----------+---------------------------------------------------------+--------------------------+
|0111301    |Cultivo de arroz                                         |2025-07-24 09:49:47.363057|
|0111302    |Cultivo de milho                                         |2025-07-24 09:49:47.363057|
|0111303    |Cultivo de trigo                                         |2025-07-24 09:49:47.363057|
|0111399    |Cultivo de outros cereais não especificados anteriormente|2025-07-24 09:49:47.363057|
|0112101    |Cultivo de algodão herbáceo                              |2025-07-24 09:49:47.363057|
+-----------+----------------------------------------------------

In [53]:
try:
    null_count(df_cnae_trusted)
except Exception as e:
    print(f"[ERRO] Falha na contagem de valores nulos: {e}")

+-----------+--------------+-------------+
|codigo_cnae|descricao_cnae|data_ingestao|
+-----------+--------------+-------------+
|          0|             0|            0|
+-----------+--------------+-------------+



## TRUSTED - Qualificações

### Etapa 1: Leitura da Camada RAW

In [58]:
df_qualificacao_raw.printSchema()
df_qualificacao_raw.show(5, truncate=False)

root
 |-- codigo_qualificacao: string (nullable = true)
 |-- descricao_qualificacao: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-------------------+----------------------------+--------------------------+
|codigo_qualificacao|descricao_qualificacao      |data_ingestao             |
+-------------------+----------------------------+--------------------------+
|00                 |Não informada               |2025-07-24 10:24:45.591468|
|05                 |Administrador               |2025-07-24 10:24:45.591468|
|08                 |Conselheiro de Administração|2025-07-24 10:24:45.591468|
|09                 |Curador                     |2025-07-24 10:24:45.591468|
|10                 |Diretor                     |2025-07-24 10:24:45.591468|
+-------------------+----------------------------+--------------------------+
only showing top 5 rows


### Etapa 2: Escrita na Camada Trusted (TRS)

In [61]:
try:
    df_qualificacao_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/qualificacoes")

    print("Dados salvos com sucesso na camada Trusted!")
except Exception as e:
    print(f"[ERRO] Falha ao salvar dados: {e}")
    raise

Dados salvos com sucesso na camada Trusted!


### Etapa 3: Ajuste e Validação

In [63]:
try:
    deltaTable_qualificacao_trusted = DeltaTable.forPath(spark, "../TRS/qualificacoes")
    df_qualificacao_trusted = deltaTable_qualificacao_trusted.toDF()

    df_qualificacao_trusted.printSchema()
    df_qualificacao_trusted.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

root
 |-- codigo_qualificacao: string (nullable = true)
 |-- descricao_qualificacao: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-------------------+----------------------------+--------------------------+
|codigo_qualificacao|descricao_qualificacao      |data_ingestao             |
+-------------------+----------------------------+--------------------------+
|00                 |Não informada               |2025-07-24 10:24:45.591468|
|05                 |Administrador               |2025-07-24 10:24:45.591468|
|08                 |Conselheiro de Administração|2025-07-24 10:24:45.591468|
|09                 |Curador                     |2025-07-24 10:24:45.591468|
|10                 |Diretor                     |2025-07-24 10:24:45.591468|
+-------------------+----------------------------+--------------------------+
only showing top 5 rows


In [64]:
try:
    null_count(df_qualificacao_trusted)
except Exception as e:
    print(f"[ERRO] Falha na contagem de nulos: {e}")

+-------------------+----------------------+-------------+
|codigo_qualificacao|descricao_qualificacao|data_ingestao|
+-------------------+----------------------+-------------+
|                  0|                     0|            0|
+-------------------+----------------------+-------------+



## TRUSTED - Empresas

###  Etapa 1: Leitura da Camada RAW

In [6]:
df_empresas_raw.printSchema()
df_empresas_raw.show(5, truncate=False)

root
 |-- cnpj_basico: string (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- codigo_qualificacao_responsavel: string (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- codigo_porte_empresa: string (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)



25/07/24 12:11:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+------------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+-------------------------+
|cnpj_basico|razao_social                                          |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social |codigo_porte_empresa|ente_federativo_responsavel|data_ingestao            |
+-----------+------------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+-------------------------+
|00000000   |BANCO DO BRASIL SA                                    |2038                    |10                             |120000000000,00|05                  |NULL                       |2025-07-24 09:59:28.28794|
|00000001   |ASSOCIACAO DE AMIGOS DE BAIRRO DO CONJ PAULISTANO     |3999                    |16                             |0,00   

### Etapa 2: Transformação dos Dados

In [7]:
# Alterando "," para "." na capital_social antes de transformal em Decimal
df_empresas_raw = df_empresas_raw.withColumn(
    "capital_social",
    f.regexp_replace("capital_social", ",", ".")
)

df_empresas_raw.printSchema()
df_empresas_raw.show(5, truncate=False)

root
 |-- cnpj_basico: string (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- codigo_qualificacao_responsavel: string (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- codigo_porte_empresa: string (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-----------+------------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+-------------------------+
|cnpj_basico|razao_social                                          |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social |codigo_porte_empresa|ente_federativo_responsavel|data_ingestao            |
+-----------+------------------------------------------------------+------------------------+-------------------------------+---------------+--------

In [8]:
try:
    null_count(df_empresas_raw)
except Exception as e:
    print(f"[ERRO] Falha na leitura de nulos")

+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+
|cnpj_basico|razao_social|codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|data_ingestao|
+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+
|          0|           0|                       0|                              0|             0|                   5|                    4487017|            0|
+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+



In [9]:
# Obter o tamanho máximo de digitos antes da "," para melhor conveção
df_max_digitos = df_empresas_raw.withColumn(
    "capital_str", f.regexp_replace(f.col("capital_social").cast("string"), "[^0-9]", "")
)

df_max_digitos.selectExpr("max(length(capital_str)) as max_digits").show()

+----------+
|max_digits|
+----------+
|        14|
+----------+



In [10]:
# Alteração de tipo para DecimalType antes de escrever no Trusted
df_empresas_raw = df_empresas_raw.withColumn(
    "capital_social",
    df_empresas_raw["capital_social"] \
        .cast(DecimalType(20, 2)) \
)

df_empresas_raw.orderBy(df_empresas_raw["capital_social"].desc()).show(10, truncate=False)
df_empresas_raw.printSchema()

+-----------+-------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+-------------------------+
|cnpj_basico|razao_social                                     |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social |codigo_porte_empresa|ente_federativo_responsavel|data_ingestao            |
+-----------+-------------------------------------------------+------------------------+-------------------------------+---------------+--------------------+---------------------------+-------------------------+
|01482657   |PAPPY RO CONFECCOES INDUSTRIA E COMERCIO LTDA    |2062                    |49                             |800000352140.77|01                  |NULL                       |2025-07-24 09:59:28.28794|
|02695531   |PRESERVI ZELADORIA RESIDENCIAL E EMPRESARIAL LTDA|2062                    |49                             |432055369515.00|01              

### Etapa 3: Escrita na Camada Trusted (TRS)

In [11]:
try:
    df_empresas_raw.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("codigo_natureza_juridica") \
        .save("../TRS/empresas")
    print("Dados salvos com sucesso na camada Trusted!")
except Exception as e:
    print(f"[ERRO] Falha ao salvar dados: {e}")
    raise

Dados salvos com sucesso na camada Trusted!


### Etapa 4: Ajuste e Validação

In [12]:
try:
    deltaTable_empresas_trusted = DeltaTable.forPath(spark, "../TRS/empresas")
    df_empresas_trusted = deltaTable_empresas_trusted.toDF()
    df_empresas_trusted.printSchema()
    df_empresas_trusted.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura: {e}")
    raise

root
 |-- cnpj_basico: string (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- codigo_qualificacao_responsavel: string (nullable = true)
 |-- capital_social: decimal(20,2) (nullable = true)
 |-- codigo_porte_empresa: string (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

+-----------+-------------------------------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------------------+
|cnpj_basico|razao_social                                           |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|data_ingestao            |
+-----------+-------------------------------------------------------+------------------------+-------------------------------+--------------+-

In [18]:
try:
    null_count(df_empresas_trusted)
except Exception as e:
    print(f"[ERRO] Falha ao verificar nulos")
    raise

+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+
|cnpj_basico|razao_social|codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|data_ingestao|
+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+
|          0|           0|                       0|                              0|             0|                   5|                    4487017|            0|
+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+



In [19]:
df_empresas_trusted.select("ente_federativo_responsavel") \
    .filter(df_empresas_trusted.ente_federativo_responsavel.isNotNull()) \
    .show(5, truncate=False)

+---------------------------+
|ente_federativo_responsavel|
+---------------------------+
|ENEAS MARQUES - PR         |
|CAMBE - PR                 |
|PORTO VITORIA - PR         |
|SANTO ANTONIO DO CAIUA - PR|
|PARANACITY - PR            |
+---------------------------+
only showing top 5 rows


In [20]:
deltaTable_empresas_trusted.update(
    condition=f.col("ente_federativo_responsavel").isNull(),
    set={
        "ente_federativo_responsavel": f.lit("desconhecido")
    }
)

25/07/24 12:22:00 WARN UpdateCommand: Could not validate number of records due to missing statistics.


In [27]:
try:
    null_count(df_empresas_trusted)
    df_empresas_trusted.select("codigo_porte_empresa") \
    .filter(df_empresas_trusted.codigo_porte_empresa.isNotNull()) \
    .show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha ao verificar nulos")
    raise

+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+
|cnpj_basico|razao_social|codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|data_ingestao|
+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+
|          0|           0|                       0|                              0|             0|                   0|                          0|            0|
+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+

+--------------------+
|codigo_porte_empresa|
+--------------------+
|01                  |
|05                  |
|05                  |
|05                  |
|05                  |
+----

In [26]:
deltaTable_empresas_trusted.update(
    condition=f.col("codigo_porte_empresa").isNull(),
    set={
        "codigo_porte_empresa": f.lit("00")
    }
)

25/07/24 12:23:55 WARN UpdateCommand: Could not validate number of records due to missing statistics.


In [29]:
# Confirmação de tratamento de nulos
try:
    null_count(df_empresas_trusted)
except Exception as e:
    print(f"[ERRO] Falha ao verificar nulos")
    raise

+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+
|cnpj_basico|razao_social|codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|data_ingestao|
+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+
|          0|           0|                       0|                              0|             0|                   0|                          0|            0|
+-----------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+-------------+



## TRUSTED - Estabelecimentos

###  Etapa 1: Leitura da Camada RAW

In [30]:
df_estabelecimentos_raw.printSchema()
df_estabelecimentos_raw.show(5, truncate=False)


root
 |-- cnpj_base: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dv: string (nullable = true)
 |-- matriz_filial: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: string (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- data_inicio_atividade: string (nullable = true)
 |-- cnae_fiscal: string (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- codigo_municipio: string (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- telefone1: string

+---------+----------+-------+-------------+-----------------------------+------------------+-----------------------+-------------------------+--------------------+----+---------------------+-----------+---------------------------------------+---------------+-------------------------+------+-----------+--------------+--------+---+----------------+----+---------+----+---------+-------+----+-----------------------------------+-----------------+----------------------+--------------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia                |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria                 |tipo_logradouro|logradouro               |numero|complemento|bairro        |cep     |uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax |email                              |situacao_especial|data_situacao_especial|data_ingestao             |
+-----

### Etapa 2: Transformação dos dados no RAW

In [31]:
# Removendo datas inválidas

try:
    df_estabelecimentos_raw = df_estabelecimentos_raw \
        .withColumn(
            "data_situacao_cadastral",
            f.when(f.length("data_situacao_cadastral") == 8,
                   f.to_date("data_situacao_cadastral", "yyyyMMdd")
                   ).otherwise(None)
        ) \
        .withColumn(
            "data_situacao_especial",
            f.when(f.length("data_situacao_especial") == 8,
                   f.to_date("data_situacao_especial", "yyyyMMdd")
                   ).otherwise(None)
        ) \
        .withColumn(
            "data_inicio_atividade",
            f.when(f.length("data_inicio_atividade") == 8,
                   f.to_date("data_inicio_atividade", "yyyyMMdd")
                   ).otherwise(None)
        )

    
except Exception as e:
    print(f"[ERRO] Falha na alteração: {e}")
    raise

In [32]:
df_estabelecimentos_raw.printSchema()
df_estabelecimentos_raw.show(5, truncate=False)

root
 |-- cnpj_base: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dv: string (nullable = true)
 |-- matriz_filial: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- data_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal: string (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- codigo_municipio: string (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- telefone1: string (nu

### Etapa 2: Escrita na Camada TRUSTED (TRS)

In [88]:
try:
    df_estabelecimentos_raw.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("uf") \
        .save("../TRS/estabelecimentos")
    
    print("Dados salvos com sucesso na camada Trusted!")
except Exception as e:
    print(f"[ERRO] Falha ao salvar dados: {e}")
    raise

[ERRO] Falha ao salvar dados: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: 5f574aaa-f232-4f66-902b-06112e5ec9db).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- cnpj_base: string (nullable = true)
-- cnpj_ordem: string (nullable = true)
-- cnpj_dv: string (nullable = true)
-- matriz_filial: string (nullable = true)
-- nome_fantasia: string (nullable = true)
-- situacao_cadastral: string (nullable = true)
-- data_situacao_cadastral: date (nullable = true)
-- motivo_situacao_cadastral: string (nullable = true)
-- nome_cidade_exterior: string (nullable = true)
-- codigo_pais: string (nullable = true)
-- data_inicio_atividade: date (nullable = true)
-- cnae_fiscal: string (n

AnalysisException: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: 5f574aaa-f232-4f66-902b-06112e5ec9db).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- cnpj_base: string (nullable = true)
-- cnpj_ordem: string (nullable = true)
-- cnpj_dv: string (nullable = true)
-- matriz_filial: string (nullable = true)
-- nome_fantasia: string (nullable = true)
-- situacao_cadastral: string (nullable = true)
-- data_situacao_cadastral: date (nullable = true)
-- motivo_situacao_cadastral: string (nullable = true)
-- nome_cidade_exterior: string (nullable = true)
-- codigo_pais: string (nullable = true)
-- data_inicio_atividade: date (nullable = true)
-- cnae_fiscal: string (nullable = t

### Etapa 3: Ajuste e Validação na TRS

In [100]:
try:
    deltaTable_estabelecimentos_trs = DeltaTable.forPath(spark, "../TRS/estabelecimentos")
    df_estabelecimentos_trs = deltaTable_estabelecimentos_trs.toDF()
    df_estabelecimentos_trs.printSchema()
    df_estabelecimentos_trs.show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura do DeltaTable: {e}")
    raise

root
 |-- cnpj_base: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dv: string (nullable = true)
 |-- matriz_filial: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- codigo_pais: string (nullable = true)
 |-- data_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal: string (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- codigo_municipio: string (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- telefone1: str

- Validação de CNPJ

In [35]:
# Concatenação dos dados para validação do CNPJ
df_cnpj = df_estabelecimentos_trs.withColumn(
    "cnpj",
    f.concat_ws("",
    "cnpj_base", "cnpj_ordem", "cnpj_dv"
    )
)

In [36]:
# Importação da biblioteca de validação
from pycpfcnpj import cpfcnpj

In [ ]:
# Função para validar
def validar_cnpj(cnpj_str):
    try:
        return "valido" if cpfcnpj.validate(cnpj_str) else "invalido"
    except:
        return "invalido"


# Registra essa função como uma UDF (User Defined Function) no Spark
validar_cnpj_udf = f.udf(validar_cnpj, StringType())

# Aplica ao DataFrame Spark
df_cnpj_validado = df_cnpj.withColumn("validacao", validar_cnpj_udf(f.col("cnpj")))


In [40]:
df_cnpj_validado.select("cnpj", "validacao").filter(df_cnpj_validado["validacao"] == "invalido").count()

0

In [41]:
try:
    null_count(df_estabelecimentos_trs)
except Exception as e:
    print(f"[ERRO] Falha na contagem de nulos: {e}")
    raise

+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+-------+---------------------+-----------+----------------------+---------------+----------+------+-----------+------+-----+---+----------------+-------+---------+-------+---------+-------+-------+-------+-----------------+----------------------+-------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|   pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|tipo_logradouro|logradouro|numero|complemento|bairro|  cep| uf|codigo_municipio|   ddd1|telefone1|   ddd2|telefone2|ddd_fax|    fax|  email|situacao_especial|data_situacao_especial|data_ingestao|
+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+-------+---------------------+----

In [ ]:
try:
# Remoção de nulos no "nome_fantasia" para "desconhecido"
    deltaTable_estabelecimentos_trs.update(
        condition=f.col("nome_fantasia").isNull(),
        set={ "nome_fantasia": f.lit("desconhecido") }
    )
except Exception as e:
    print(f"[ERRO] Falha no update: {e}")
    raise

25/07/24 12:45:26 WARN UpdateCommand: Could not validate number of records due to missing statistics.


In [45]:
# filtragem para encontrar erro na quantidade de caracteres de dados como UF, cnpj_base etc
df_estabelecimentos_trs.select("nome_cidade_exterior").filter(
    df_estabelecimentos_trs["nome_cidade_exterior"].isNotNull()
).show(5, truncate=False)

+--------------------+
|nome_cidade_exterior|
+--------------------+
|TOQUIO              |
|ILLIINOIS           |
|MONTEVIDEU          |
|AMSTERDA            |
|ROMA                |
+--------------------+
only showing top 5 rows


In [ ]:
try:
    # Remoção de nulos no "nome_cidade_exterior" para "desconhecido"
    deltaTable_estabelecimentos_trs.update(
        condition=f.col("nome_cidade_exterior").isNull(),
        set={ "nome_cidade_exterior": f.lit("desconhecido") }
    )
except Exception as e:
    print(f"[ERRO] Falha no update: {e}")
    raise

25/07/24 12:48:12 WARN UpdateCommand: Could not validate number of records due to missing statistics.


In [53]:
try:
    df_estabelecimentos_trs.select("pais").filter(
        df_estabelecimentos_trs["pais"].isNotNull()
    ).show(5, truncate=False)
except Exception as e:
    print(f"[ERRO] Falha na leitura: {e}")
    raise

+----+
|pais|
+----+
|105 |
|105 |
|105 |
|105 |
|105 |
+----+
only showing top 5 rows


In [84]:
df_estabelecimentos_trs = df_estabelecimentos_trs.withColumnRenamed("codig_pais", "codigo_pais")

In [85]:
try:
    df_estabelecimentos_trs.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", True) \
        .save("../TRS/estabelecimentos")
    
except Exception as e:
    print(f"[ERRO] Falha na sobreescrita: {e}")
    raise

In [86]:
df_estabelecimentos_trs.printSchema()

root
 |-- cnpj_base: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dv: string (nullable = true)
 |-- matriz_filial: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- codigo_pais: string (nullable = true)
 |-- data_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal: string (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- codigo_municipio: string (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- telefone1: str

In [54]:
try:
    # Remoção de nulos no "nome_cidade_exterior" para "desconhecido"
    deltaTable_estabelecimentos_trs.update(
        condition=f.col("pais").isNull(),
        set={ "pais": f.lit("000") }
    )
except Exception as e:
    print(f"[ERRO] Falha no update: {e}")
    raise

25/07/24 12:53:52 WARN UpdateCommand: Could not validate number of records due to missing statistics.


In [ ]:
try:
    # Remoção de nulos no endereço, substituindo por ""
    deltaTable_estabelecimentos_trs.update(
        condition=(
            f.col("fax") == "00000000"

        ),
        set={
            "ddd_fax": f.lit("00"),
        }
    )
except Exception as e:
    print(f"[ERRO] Falha no update: {e}")
    raise

In [ ]:
try:
    null_count(df_estabelecimentos_trs)
except Exception as e:
    print(f"[ERRO] Falha na contagem de nulos: {e}")
    raise

+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+----------------------+---------------+----------+------+-----------+------+---+---+----------------+-------+---------+-------+---------+-------+---+-------+-----------------+----------------------+-------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|codigo_pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|tipo_logradouro|logradouro|numero|complemento|bairro|cep| uf|codigo_municipio|   ddd1|telefone1|   ddd2|telefone2|ddd_fax|fax|  email|situacao_especial|data_situacao_especial|data_ingestao|
+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+----

In [132]:
df_estabelecimentos_trs.select("cnae_fiscal_secundaria").filter(df_estabelecimentos_trs.codigo_municipio != "").orderBy(df_estabelecimentos_trs.codigo_municipio).show()

+----------------------+
|cnae_fiscal_secundaria|
+----------------------+
|               0000000|
|               0000000|
|               0000000|
|               0000000|
|               0000000|
|  7112000,4313400,4...|
|               0000000|
|               0000000|
|               0000000|
|       1412602,4755502|
|               0000000|
|               0000000|
|  4755503,4755502,4...|
|  4712100,4721103,4...|
|  8130300,5021101,4...|
|               0000000|
|  4645101,4646001,4...|
|  4781400,4772500,4...|
|  4541206,4753900,4...|
|               4543900|
+----------------------+
only showing top 20 rows


In [41]:
# Efetuar updates em dados errados

try:
    # update por CNPJ
    deltaTable_estabelecimentos_trs.update(
        condition= (f.col("cnpj_base") == "29744778")
            & (f.col("cnpj_ordem") == "4750")
            & (f.col("cnpj_dv") == "39"),
        set={
            # "nome_fantasia": f.lit("BONGIOVANI"),
            # "numero": f.lit("200"),
            "complemento": f.lit("ESQUINA COM RUA GABRIELA RODRIGUES"),
            # "bairro": f.lit("CENTRO"),
            # "cep": f.lit("78937000"),
            # "uf": f.lit("RO"),
            # "codigo_municipio": f.lit("0683"),
        }
    )

    # deltaTable_estabelecimentos_trs.update(
    #     condition=(
    #         f.col("telefone2").isNull()
    #     ),
    #     set={
    #         "ddd2": f.lit(None),
    #     }
    # )
    print("Atualização efetuada!")
except Exception as e:
    print(f"Erro: {e}")

25/07/20 14:31:32 WARN UpdateCommand: Could not validate number of records due to missing statistics.


Atualização efetuada!


In [15]:
# Alteração no nome de colunas
try:
    df_estabelecimentos_trs = df_estabelecimentos_trs.withColumnRenamed("pais", "codigo_pais")
    df_estabelecimentos_trs.show(5, truncate=False)
    df_estabelecimentos_trs.printSchema()
except Exception as e:
    print(f"Erro: {e}")

+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+---------------------------------------------------------------------------------------+---------------+---------------------------+------+-----------+----------------+--------+---+----------------+----+---------+----+---------+-------+----+----------------------------+-----------------+----------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|codigo_pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria                                                                 |tipo_logradouro|logradouro                 |numero|complemento|bairro          |cep     |uf |codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax |email                       |situacao_especial|data_situaca

In [ ]:
# Cotagem de valores nulos por coluna

try:
    nulos_df = df_estabelecimentos_trs.select([
        f.sum(f.col(c).isNull().cast("int")).alias(c)
        for c in df_estabelecimentos_trs.columns
    ])

    nulos_df.selectExpr("stack(" + str(len(nulos_df.columns)) + ", " +
                        ", ".join([f"'{c}', {c}" for c in nulos_df.columns]) +
                        ") as (coluna, total_nulos)") \
            .orderBy(f.desc("total_nulos")) \
            .show(truncate=False, vertical=True)
except Exception as e:
    print(f"Erro: {e}")

-RECORD 0------------------------------
 coluna      | ddd1                    
 total_nulos | 4753435                 
-RECORD 1------------------------------
 coluna      | telefone1               
 total_nulos | 4753435                 
-RECORD 2------------------------------
 coluna      | ddd2                    
 total_nulos | 4753435                 
-RECORD 3------------------------------
 coluna      | telefone2               
 total_nulos | 4753435                 
-RECORD 4------------------------------
 coluna      | nome_cidade_exterior    
 total_nulos | 4751302                 
-RECORD 5------------------------------
 coluna      | situacao_especial       
 total_nulos | 4750384                 
-RECORD 6------------------------------
 coluna      | data_situacao_especial  
 total_nulos | 4750384                 
-RECORD 7------------------------------
 coluna      | codigo_pais             
 total_nulos | 4601806                 
-RECORD 8------------------------------


In [16]:
# Atualizar dados do delta
try:
    df_estabelecimentos_trs.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .partitionBy("uf") \
        .save("../TRS/estabelecimentos")
except Exception as e:
    print(f"Erro: {e}")

## TRUSTED - Sócios

### Etapa 1: Leitura da Camada RAW

In [ ]:
try:
    deltaTable_socios_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/socios")
    df_socios_raw: DataFrame = deltaTable_socios_raw.toDF()
    df_socios_raw.show(5, truncate=False)
    df_socios_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+-------------------+------------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social             |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+------------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|15508939   |2                  |LUIZ FERNANDO RODRIGUES DE SALES    |***784978**   |49                       |20120423              |NULL|***000000**        |NULL              |00                               |5           |
|15508939   |2                  |LUIZ GONZAGA RODRIGUES DE SALES     |***555608**   |49         

### Etapa 2: Transformação dos dados

In [46]:
try:
    # Verificar se existe divergencia no tamanho das datas
    df_socios_raw.filter(f.length(df_socios_raw["data_entrada_sociedade"]) != 8).show()
except Exception as e:
    print(f"Erro: {e}")

+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social|cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+



In [47]:
try:
    df_socios_raw = df_socios_raw.withColumn(
        "data_entrada_sociedade",
        f.to_date(df_socios_raw["data_entrada_sociedade"], "yyyyMMdd")
    )
    df_socios_raw.show(truncate=False)
    print(df_socios_raw.select("cnpj_basico").count())
except Exception as e:
    print(f"Erro: {e}")

+-----------+-------------------+------------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social             |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|pais|representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+------------------------------------+--------------+-------------------------+----------------------+----+-------------------+------------------+---------------------------------+------------+
|15508939   |2                  |LUIZ FERNANDO RODRIGUES DE SALES    |***784978**   |49                       |2012-04-23            |NULL|***000000**        |NULL              |00                               |5           |
|15508939   |2                  |LUIZ GONZAGA RODRIGUES DE SALES     |***555608**   |49         

### Etapa 3: Escrita na Camada Trusted

In [48]:
try:
    df_socios_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/socios")
    print("Escrito com sucesso!")
except Exception as e:
    print(f"Erro na escrita: {e}")

Escrito com sucesso!


### Etapa 4: Ajuste e Validação

In [54]:
# confirmar como esta o delta no TRS
try:
    deltaTable_socios_trusted: DeltaTable = DeltaTable.forPath(spark, "../TRS/socios")
    df_socios_trusted = deltaTable_socios_trusted.toDF()
    df_socios_trusted.show(5, truncate=False)
    df_socios_trusted.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+-------------------+-----------------------------+--------------+-------------------------+----------------------+------------------+----------------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social      |cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|socios_codigo_pais|cnpj_cpf_representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+-----------------------------+--------------+-------------------------+----------------------+------------------+----------------------------+------------------+---------------------------------+------------+
|03123458   |2                  |JOSE CERQUEIRA COSTA         |***445205**   |49                       |1999-04-30            |NULL              |***000000**                 |NULL              |00                               |6           |
|03123458   |2                  

In [50]:
# Ajuste no nome das colunas
df_socios_trusted = df_socios_trusted.withColumnRenamed(
    "pais",
    "socios_codigo_pais"
).withColumnRenamed(
    "representante_legal",
    "cnpj_cpf_representante_legal"
)

df_socios_trusted.show()

+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+------------------+----------------------------+------------------+---------------------------------+------------+
|cnpj_basico|identificador_socio|nome_socio_razao_social|cnpj_cpf_socio|codigo_qualificacao_socio|data_entrada_sociedade|socios_codigo_pais|cnpj_cpf_representante_legal|nome_representante|codigo_qualificacao_representante|faixa_etaria|
+-----------+-------------------+-----------------------+--------------+-------------------------+----------------------+------------------+----------------------------+------------------+---------------------------------+------------+
|   15508939|                  2|   LUIZ FERNANDO ROD...|   ***784978**|                       49|            2012-04-23|              NULL|                 ***000000**|              NULL|                               00|           5|
|   15508939|                  2|   LUIZ GONZAGA RODR...

In [57]:
try:
    nulos_df = df_socios_trusted.select([
        f.sum(f.col(c).isNull().cast("int")).alias(c)
        for c in df_socios_trusted.columns
    ])

    nulos_df.selectExpr("stack(" + str(len(nulos_df.columns)) + ", " +
                        ", ".join([f"'{c}', {c}" for c in nulos_df.columns]) +
                        ") as (coluna, total_nulos)") \
            .orderBy(f.desc("total_nulos")) \
            .show(truncate=False)
except Exception as e:
    print(f"Erro: {e}")


+---------------------------------+-----------+
|coluna                           |total_nulos|
+---------------------------------+-----------+
|socios_codigo_pais               |2012151    |
|nome_representante               |1969517    |
|cnpj_cpf_socio                   |1164       |
|nome_socio_razao_social          |173        |
|cnpj_basico                      |0          |
|identificador_socio              |0          |
|codigo_qualificacao_socio        |0          |
|data_entrada_sociedade           |0          |
|cnpj_cpf_representante_legal     |0          |
|codigo_qualificacao_representante|0          |
|faixa_etaria                     |0          |
+---------------------------------+-----------+



In [52]:
# Atualizar dados do delta
try:
    df_socios_trusted.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .save("../TRS/socios")
except Exception as e:
    print(f"Erro: {e}")

## TRUSTED - Simples Nacional

### Etapa 1: Leitura da Camada RAW

In [60]:
try:
    deltaTable_simples_nacional_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/simples_nacional")
    df_simples_nacional_raw = deltaTable_simples_nacional_raw.toDF()
    df_simples_nacional_raw.show(5, truncate=False)
    df_simples_nacional_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|36536474   |N            |20200303          |20201231             |N        |00000000      |00000000         |
|36536475   |S            |20200303          |00000000             |S        |20200303      |00000000         |
|36536476   |N            |20200303          |20220331             |N        |20200303      |20220331         |
|36536477   |N            |20200303          |20200403             |N        |20200303      |20200403         |
|36536478   |N            |20200303          |20241231             |N        |20200303      |20241231         |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------

### Etapa 2: Transformação dos dados

In [61]:
# Tratando a data antes de passar para DateType

try:
    df_simples_nacional_raw = df_simples_nacional_raw.withColumn(
        "data_exclusao_simples", f.when(
            df_simples_nacional_raw["data_exclusao_simples"] == "00000000",
            None
        ).otherwise(df_simples_nacional_raw["data_exclusao_simples"])
    ).withColumn(
        "data_opcao_simples", f.when(
            df_simples_nacional_raw["data_opcao_simples"] == "00000000",
            None
        ).otherwise(df_simples_nacional_raw["data_opcao_simples"])
    ).withColumn(
        "data_opcao_mei", f.when(
            df_simples_nacional_raw["data_opcao_mei"] == "00000000",
            None
        ).otherwise(df_simples_nacional_raw["data_opcao_mei"])
    ).withColumn(
        "data_exclusao_mei", f.when(
            df_simples_nacional_raw["data_exclusao_mei"] == "00000000",
            None
        ).otherwise(df_simples_nacional_raw["data_exclusao_mei"])
    )

    print(df_simples_nacional_raw.filter(f.length(df_simples_nacional_raw["data_exclusao_simples"]) != 8).count())
    print(df_simples_nacional_raw.filter(f.length(df_simples_nacional_raw["data_opcao_simples"]) != 8).count())
    print(df_simples_nacional_raw.filter(f.length(df_simples_nacional_raw["data_opcao_mei"]) != 8).count())
    print(df_simples_nacional_raw.filter(f.length(df_simples_nacional_raw["data_exclusao_mei"]) != 8).count())
except Exception as e:
    print(f"Erro: {e}")

0


0


0


0


In [62]:
try:
    df_simples_nacional_raw = df_simples_nacional_raw.withColumn(
        "data_opcao_simples", 
        f.to_date(df_simples_nacional_raw["data_opcao_simples"], "yyyyMMdd")
    ).withColumn(
        "data_exclusao_simples", 
        f.to_date(df_simples_nacional_raw["data_exclusao_simples"], "yyyyMMdd")
    ).withColumn(
        "data_opcao_mei", 
        f.to_date(df_simples_nacional_raw["data_opcao_mei"], "yyyyMMdd")
    ).withColumn(
        "data_exclusao_mei", 
        f.to_date(df_simples_nacional_raw["data_exclusao_mei"], "yyyyMMdd")
    )

    df_simples_nacional_raw.show(5, truncate=False)
except Exception as e:
    print(f"Erro: {e}")

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|36536474   |N            |2020-03-03        |2020-12-31           |N        |NULL          |NULL             |
|36536475   |S            |2020-03-03        |NULL                 |S        |2020-03-03    |NULL             |
|36536476   |N            |2020-03-03        |2022-03-31           |N        |2020-03-03    |2022-03-31       |
|36536477   |N            |2020-03-03        |2020-04-03           |N        |2020-03-03    |2020-04-03       |
|36536478   |N            |2020-03-03        |2024-12-31           |N        |2020-03-03    |2024-12-31       |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------

### Etapa 3: Escrita na camada Trusted (TRS)

In [ ]:
try:
    df_simples_nacional_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/simples_nacional")
    print("Dados escrito com sucesso!")
except Exception as e:
    print(f"Erro ao gravar dados: {e}")

### Etapa 4: Ajuste e Validação

In [64]:
# Confirmar alterações, pela camada TRS
try:
    deltaTable_simples_nacional_trusted: DeltaTable = DeltaTable.forPath(spark, "../TRS/simples_nacional")
    df_simples_nacional_trusted = deltaTable_simples_nacional_trusted.toDF()
    df_simples_nacional_trusted.show(5, truncate=False)
    df_simples_nacional_trusted.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_basico|opcao_simples|data_opcao_simples|data_exclusao_simples|opcao_mei|data_opcao_mei|data_exclusao_mei|
+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|23331889   |N            |2015-09-23        |2023-12-31           |N        |2015-09-23    |2023-12-31       |
|23331890   |S            |2015-09-23        |NULL                 |S        |2015-09-23    |NULL             |
|23331891   |N            |2015-09-23        |2017-07-25           |N        |2015-09-23    |2017-07-25       |
|23331892   |N            |2018-01-01        |2019-12-31           |N        |NULL          |NULL             |
|23331893   |N            |2015-09-23        |2016-05-30           |N        |2015-09-23    |2016-05-30       |
+-----------+-------------+------------------+---------------------+---------+--------------+-----------

## TRUSTED - Motivos

### Etapa 1: Leitura da Camada RAW

In [65]:
try:
    deltaTable_motivos_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/motivos")
    df_motivos_raw: DataFrame = deltaTable_motivos_raw.toDF()
    df_motivos_raw.show(63, truncate=False)
    df_motivos_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-------------+--------------------------------------------------------------------------------------+
|codigo_motivo|descricao_motivo                                                                      |
+-------------+--------------------------------------------------------------------------------------+
|00           |SEM MOTIVO                                                                            |
|01           |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA                                       |
|02           |INCORPORACAO                                                                          |
|03           |FUSAO                                                                                 |
|04           |CISAO TOTAL                                                                           |
|05           |ENCERRAMENTO DA FALENCIA                                                              |
|06           |ENCERRAMENTO DA LIQUIDACAO                                

### Etapa 2: Escrita na camada Trusted (TRS)

In [66]:
try:
    df_motivos_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/motivos")
except Exception as e:
    print(f"Erro ao gravar dados: {e}")

### Etapa 3: Ajuste e Validação

In [67]:
try:
    deltaTable_motivos_trusted: DeltaTable = DeltaTable.forPath(spark, "../TRS/motivos")
    df_motivos_trusted = deltaTable_motivos_trusted.toDF()
    df_motivos_trusted.show(5, truncate=False)
    df_motivos_trusted.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-------------+-----------------------------------------------+
|codigo_motivo|descricao_motivo                               |
+-------------+-----------------------------------------------+
|00           |SEM MOTIVO                                     |
|01           |EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA|
|02           |INCORPORACAO                                   |
|03           |FUSAO                                          |
|04           |CISAO TOTAL                                    |
+-------------+-----------------------------------------------+
only showing top 5 rows
root
 |-- codigo_motivo: string (nullable = true)
 |-- descricao_motivo: string (nullable = true)



## TRUSTED - Paises

### Etapa 1: Leitura da Camada RAW

In [68]:
try:
    deltaTable_paises_raw: DeltaTable = DeltaTable.forPath(spark, "../RAW/paises")
    df_paises_raw = deltaTable_paises_raw.toDF()
    df_paises_raw.show(5, truncate=False)
    print(f"Total de Paises: {df_paises_raw.count()}")
    df_paises_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+------------------------+---------------------+
|codigo_natureza_juridica|natureza_juridica    |
+------------------------+---------------------+
|000                     |COLIS POSTAUX        |
|013                     |AFEGANISTAO          |
|017                     |ALBANIA              |
|020                     |ALBORAN-PEREJIL,ILHAS|
|023                     |ALEMANHA             |
+------------------------+---------------------+
only showing top 5 rows
Total de Paises: 255
root
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- natureza_juridica: string (nullable = true)



### Etapa 2: Transformação dos dados

In [69]:
try:
    df_paises_raw = df_paises_raw.withColumnsRenamed(
        {
            "codigo_natureza_juridica": "codigo_pais",
            "natureza_juridica": "pais"
        },
    )

    df_paises_raw.show(5, truncate=False)
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+---------------------+
|codigo_pais|pais                 |
+-----------+---------------------+
|000        |COLIS POSTAUX        |
|013        |AFEGANISTAO          |
|017        |ALBANIA              |
|020        |ALBORAN-PEREJIL,ILHAS|
|023        |ALEMANHA             |
+-----------+---------------------+
only showing top 5 rows


### Etapa 3: Escrita na camada Trusted (TRS)

In [70]:
try:
    df_paises_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/paises")
    print("Dados salvo com sucesso!")
except Exception as e:
    print(f"Erro na leitura: {e}")

Dados salvo com sucesso!


### Etapa 4: Ajuste e Validação

In [71]:
try:
    deltaTable_paises_trusted: DeltaTable = DeltaTable.forPath(spark, "../TRS/paises")
    df_paises_trusted = deltaTable_paises_trusted.toDF()
    df_paises_trusted.show(5, truncate=False)
    print(f"Total de Paises: {df_paises_raw.count()}")
    df_paises_raw.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+-----------+---------------------+
|codigo_pais|pais                 |
+-----------+---------------------+
|000        |COLIS POSTAUX        |
|013        |AFEGANISTAO          |
|017        |ALBANIA              |
|020        |ALBORAN-PEREJIL,ILHAS|
|023        |ALEMANHA             |
+-----------+---------------------+
only showing top 5 rows
Total de Paises: 255
root
 |-- codigo_pais: string (nullable = true)
 |-- pais: string (nullable = true)



## TRUSTED - Municipios Receita Federal

### Leitura da camada RAW

In [72]:
# Carrega dados da camada RAW
deltaTable_municipios_rf = DeltaTable.forPath(spark, "../RAW/municipios_rf")
df_municipios_rf_raw = deltaTable_municipios_rf.toDF()
df_municipios_rf_raw.show(truncate=False)

+------------+------------------------+
|id_municipio|municipio               |
+------------+------------------------+
|0001        |GUAJARA-MIRIM           |
|0002        |ALTO ALEGRE DOS PARECIS |
|0003        |PORTO VELHO             |
|0004        |BURITIS                 |
|0005        |JI-PARANA               |
|0006        |CHUPINGUAIA             |
|0007        |ARIQUEMES               |
|0008        |CUJUBIM                 |
|0009        |CACOAL                  |
|0010        |NOVA UNIAO              |
|0011        |PIMENTA BUENO           |
|0012        |PARECIS                 |
|0013        |VILHENA                 |
|0014        |PIMENTEIRAS DO OESTE    |
|0015        |JARU                    |
|0016        |PRIMAVERA DE RONDONIA   |
|0017        |OURO PRETO DO OESTE     |
|0018        |SAO FELIPE D'OESTE      |
|0019        |PRESIDENTE MEDICI       |
|0020        |SAO FRANCISCO DO GUAPORE|
+------------+------------------------+
only showing top 20 rows


### Escrita na camada Trusted (TRS)

In [73]:
try:
    df_municipios_rf_raw.write.format("delta") \
        .mode("overwrite") \
        .save("../TRS/municipios_rf")
    print("Dados salvo!")
except Exception as e:
    print(e)

Dados salvo!


In [74]:
try:
    deltaTable_municipios_rf: DeltaTable = DeltaTable.forPath(spark, "../TRS/municipios_rf")
    df_municipios_rf_trusted: DataFrame = deltaTable_municipios_rf.toDF()
    df_municipios_rf_trusted.show(5, truncate=False)
    df_municipios_rf_trusted.printSchema()
except Exception as e:
    print(f"Erro: {e}")

+------------+-----------------------+
|id_municipio|municipio              |
+------------+-----------------------+
|0001        |GUAJARA-MIRIM          |
|0002        |ALTO ALEGRE DOS PARECIS|
|0003        |PORTO VELHO            |
|0004        |BURITIS                |
|0005        |JI-PARANA              |
+------------+-----------------------+
only showing top 5 rows
root
 |-- id_municipio: string (nullable = true)
 |-- municipio: string (nullable = true)



## TRUSTED - Joins para tabela fato

In [81]:
# Recuperando as delta table

deltaTable_estabelecimentos: DeltaTable = DeltaTable.forPath(spark, "../TRS/estabelecimentos")
deltaTable_empresas: DeltaTable = DeltaTable.forPath(spark, "../TRS/empresas")
deltaTable_municipios: DeltaTable = DeltaTable.forPath(spark, "../TRS/municipios_rf")
deltaTable_municipios_ibge: DeltaTable = DeltaTable.forPath(spark, "../TRS/municipios")
deltaTable_cnae: DeltaTable = DeltaTable.forPath(spark, "../TRS/cnae")
deltaTable_natureza_juridica: DeltaTable = DeltaTable.forPath(spark, "../TRS/natureza_juridica")
deltaTable_simples_nacional: DeltaTable = DeltaTable.forPath(spark, "../TRS/simples_nacional")
deltaTable_socios: DeltaTable = DeltaTable.forPath(spark, "../TRS/socios")
deltaTable_qualificacao_socios: DeltaTable = DeltaTable.forPath(spark, "../TRS/qualificacao_socios")
deltaTable_paises: DeltaTable = DeltaTable.forPath(spark, "../TRS/paises")
deltaTable_motivos: DeltaTable = DeltaTable.forPath(spark, "../TRS/motivos")

In [82]:
# Transformando em DataFrame

df_estabelecimentos: DataFrame = deltaTable_estabelecimentos.toDF()
df_empresas: DataFrame = deltaTable_empresas.toDF()
df_municipios_rf: DataFrame = deltaTable_municipios.toDF()
df_municipios_ibge: DataFrame = deltaTable_municipios_ibge.toDF()
df_cnae: DataFrame = deltaTable_cnae.toDF()
df_natureza: DataFrame = deltaTable_natureza_juridica.toDF()
df_simples: DataFrame = deltaTable_simples_nacional.toDF()
df_socios: DataFrame = deltaTable_socios.toDF()
df_qualificacao_socios: DataFrame = deltaTable_qualificacao_socios.toDF()
df_paises: DataFrame = deltaTable_paises.toDF()
df_motivos: DataFrame = deltaTable_motivos.toDF()

In [ ]:
# Analise do Schema para evitar campos com valores iguais

df_empresas.printSchema()
df_natureza.printSchema()

root
 |-- cnpj_basico: string (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- codigo_qualificacao_responsavel: string (nullable = true)
 |-- capital_social: decimal(20,2) (nullable = true)
 |-- codigo_porte_empresa: string (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)

root
 |-- codigo_natureza_juridica: string (nullable = true)
 |-- natureza_juridica: string (nullable = true)



In [ ]:
# join da tabela empresas com natureza juridica para ter acesso a descrição de natureza juridica
try:
    df_empresas_joined_natureza_juridica: DataFrame = (
        df_empresas.join(
            df_natureza.select(
                "natureza_juridica",
                "codigo_natureza_juridica"
            ),
            df_empresas.codigo_natureza_juridica == df_natureza.codigo_natureza_juridica,
            how="left"
        )
    ).drop(
        df_natureza.codigo_natureza_juridica
    ).withColumnRenamed(
        "natureza_juridica",
        "descricao_natureza_juridica"
    )
except Exception as e:
    print(f"Erro no join: {e}")

In [88]:
# Resultado do join anterior

df_empresas_joined_natureza_juridica.show(2, truncate=False)
df_empresas_joined_natureza_juridica.printSchema()

+-----------+---------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+---------------------------+
|cnpj_basico|razao_social                     |codigo_natureza_juridica|codigo_qualificacao_responsavel|capital_social|codigo_porte_empresa|ente_federativo_responsavel|descricao_natureza_juridica|
+-----------+---------------------------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+---------------------------+
|06942044   |JOSE FRANCISCO BEZERRA TRANSPORTE|2135                    |50                             |0.00          |01                  |NULL                       |Empresário (Individual)    |
|06942045   |ADEMIR A. DA SILVA               |2135                    |50                             |0.00          |01                  |NULL                       |Empresário (Individual)    |
+-----------+--

In [96]:
# Construção da tabela fato conforme solicitado

df_tabela_fato: DataFrame = (
    df_estabelecimentos.join(
        df_cnae,
        df_estabelecimentos.cnae_fiscal == df_cnae.codigo_cnae,
        how="left"
    ).drop(df_cnae.codigo_cnae).join(
        df_municipios_rf,
        df_estabelecimentos.codigo_municipio == df_municipios_rf.id_municipio,
        how="left"
    ).drop(df_municipios_rf.id_municipio).join(
        df_empresas_joined_natureza_juridica,
        df_estabelecimentos.cnpj_base == df_empresas_joined_natureza_juridica.cnpj_basico,
        how='left'
    ).drop(
        df_empresas_joined_natureza_juridica.cnpj_basico
    ).join(
        df_simples,
        df_estabelecimentos.cnpj_base == df_simples.cnpj_basico,
        how="left"
    ).drop(df_simples.cnpj_basico)
)

In [ ]:
# Visualização do Schema final

df_tabela_fato.printSchema()

root
 |-- cnpj_base: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dv: string (nullable = true)
 |-- matriz_filial: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- codigo_pais: string (nullable = true)
 |-- data_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal: string (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- codigo_municipio: string (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- telefone1: str

In [ ]:
# Resultado visual dos dados

df_tabela_fato.printSchema()

4753435

In [100]:
# Salvar tabela fato para analise posterior
# a função "criar_tabela_fato" serve apenas para evitar criar novamente de forma acidental

def criar_tabela_fato():
    try:
        df_tabela_fato.write.format("delta") \
            .mode("overwrite") \
            .partitionBy("uf") \
            .save("../TRS/fato_cnpj")
        
        print("Dados salvos com sucesso!")
    except Exception as e:
        print(f"Erro ao gravar: {e}")
    
criar_tabela_fato()



Dados salvos com sucesso!


## Viagem no tempo (Time Travel)

In [102]:
deltaTable_estabelecimentos.history().show(truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 17                                                                                                                                                                                                                                                                                                                                                        
 timestamp           | 2025-07-20 14:31:32.288                                                                                                                                                                                                                      

In [ ]:

# Ver versões espeicificas
df_restore = spark.read.format("delta") \
    .option("versionAsOf", 6) \
    .load("../TRS/estabelecimentos")
df_restore.filter(df_restore["cnpj_ordem"].isNull() & df_restore["cnae_fiscal"].isNotNull()).show()

+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+----------------------+---------------+----------+------+-----------+------+---+---+----------------+----+---------+----+---------+-------+---+-----+-----------------+----------------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|codigo_pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|tipo_logradouro|logradouro|numero|complemento|bairro|cep| uf|codigo_municipio|ddd1|telefone1|ddd2|telefone2|ddd_fax|fax|email|situacao_especial|data_situacao_especial|
+---------+----------+-------+-------------+-------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+----------------------+-------------

In [37]:
# Restaurar versão

deltaTable_estabelecimentos_trs.restoreToVersion(6)

25/07/20 01:00:28 WARN DAGScheduler: Broadcasting large task binary with size 1087.5 KiB


DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

## Finalizar Spark

In [70]:
spark.stop()